<a href="https://colab.research.google.com/github/noambassat/SupremeCourtClassifier/blob/main/LLM_Court.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import torch
import joblib
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
import time
import re
import openai
import json
from sklearn.model_selection import train_test_split
import ast

In [2]:

drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:

folder_id = "בית המשפט - קבצים שנבדקו"
directory_path = f"/content/drive/My Drive/{folder_id}"
classifiers_path = "/content/drive/MyDrive/בית המשפט - מסווגים/"

if not os.path.exists(directory_path):
    print(f"Directory {directory_path} does not exist. Please check the folder path.")
else:
    dataframes = []

    for file_name in os.listdir(directory_path):
        if file_name.endswith(".xlsx") or file_name.endswith(".xls"):
            file_path = os.path.join(directory_path, file_name)
            try:
                df = pd.read_excel(file_path)
                dataframes.append(df)
            except Exception as e:
                print(f"Failed to read {file_name}: {e}")

    if dataframes:
        combined_df = pd.concat(dataframes, ignore_index=True)
        print("All Excel files have been concatenated successfully!")

/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


All Excel files have been concatenated successfully!


In [4]:
len(dataframes)

6

In [5]:
df_1 = dataframes[0]

df_1.columns

Index(['level_0', 'index', 'גוף המסמך', 'מספר הליך', 'שם הליך',
       'שנת פתיחת ההליך', 'גוף המסמך חתוך', 'predicted_probability',
       'prediction_original_model', 'בקשה לרשות ערעור התקבלה?',
       'טקסט שמצביע על תוצאת בקשה שונה', 'הערעור התקבל?',
       'רע"פ בקשה אחרת או דלמטה מיוחד', 'קידוד', 'קידוד חוזר', 'הערות ',
       'הערות קידוד חוזר'],
      dtype='object')

In [6]:
df_1 = df_1[['גוף המסמך', 'מספר הליך', 'שם הליך',
       'שנת פתיחת ההליך', 'גוף המסמך חתוך',  'בקשה לרשות ערעור התקבלה?',
       'טקסט שמצביע על תוצאת בקשה שונה', 'הערעור התקבל?',
       'רע"פ בקשה אחרת או דלמטה מיוחד', 'הערות ',
       'הערות קידוד חוזר']]

In [7]:
df_2 = dataframes[1]

In [8]:
df_2 = df_2[['גוף המסמך', 'מספר הליך', 'שם הליך',
       'שנת פתיחת ההליך', 'גוף המסמך חתוך',
       'בקשה לרשות ערעור התקבלה?',
       'טקסט שמצביע על תוצאת בקשה שונה (טקסטים שונים מופרדים ב-**)',
       'הערעור התקבל?', 'רע"פ בקשה אחרת או דלמטה מיוחד',
       'הערות תוצאת ערעור', 'הערות ', 'הערות קידוד חוזר']]

In [9]:
df = pd.concat([df_1, df_2], ignore_index=True)

In [10]:
df_3 = dataframes[2]

In [11]:
df_3 = df_3[['גוף המסמך', 'מספר הליך', 'שם הליך',
       'שנת פתיחת ההליך',
       'בקשה לרשות ערעור התקבלה?',
       'טקסט שמצביע על תוצאת בקשה שונה (טקסטים שונים מופרדים ב-**)',
       'הערעור התקבל?', 'הערות תוצאת ערעור', 'רע"פ בקשה אחרת או דלמטה מיוחד', 'הערות ']]

In [12]:
df = pd.concat([df, df_3], ignore_index=True)

In [13]:
columns_to_check = [
    "הערות ",
    "הערות תוצאת ערעור",
    "הערות קידוד חוזר",
    "טקסט שמצביע על תוצאת בקשה שונה",
    "טקסט שמצביע על תוצאת בקשה שונה (טקסטים שונים מופרדים ב-**)",
    "טקסט שמצביע על תוצאת בקשה שונה (טקסטים שונים מופרדים ב-**)"
]


df = df[~df[columns_to_check].isna().all(axis=1)]

In [14]:
df.shape

(348, 13)

In [15]:
len(df["גוף המסמך"])

# בדיקת האורך של כל פסק דין (במילים)
df["length"] = df["גוף המסמך"].apply(lambda x: len(str(x).split()))

# חישוב האורך הממוצע
average_length = df["length"].mean()
max_length = df["length"].max()
min_length = df["length"].min()

print(f"אורך ממוצע של פסקי דין: {average_length:.2f} מילים")
print(f"פסק הדין הארוך ביותר מכיל {max_length} מילים")
print(f"פסק הדין הקצר ביותר מכיל {min_length} מילים")


אורך ממוצע של פסקי דין: 722.41 מילים
פסק הדין הארוך ביותר מכיל 5920 מילים
פסק הדין הקצר ביותר מכיל 60 מילים


In [16]:
def convert_list_to_string(text):
    if isinstance(text, str) and text.startswith("[") and text.endswith("]"):
        try:
            text_list = ast.literal_eval(text)
            return ' '.join(text_list)
        except (ValueError, SyntaxError):
            return text
    return text

def clean_text(text):
    if isinstance(text, list):
        text = ' '.join(text)
    elif isinstance(text, str):
        text = re.sub(r'\n+', ' ', text)  # הסרת שורות חדשות מרובות
        text = re.sub(r'\n', '', text)   # הסרת תווי newline \n מהטקסט
        text = re.sub(r'\xa0', ' ', text)  # הסרת תווי \xa0 מהטקסט
        text = re.sub(r'\s+', ' ', text)  # הסרת רווחים מרובים

        text = re.sub(r"העותק כפוף לשינויי עריכה וניסוח.*?$", "", text, flags=re.MULTILINE)
        text = re.sub(r"מרכז מידע, טל' \d{2,3}-\d{6,7}.*?$", "", text, flags=re.MULTILINE)
        text = re.sub(r"אתר אינטרנט, .*?$", "", text, flags=re.MULTILINE)
    return text

print("שורות לא קריאות לפני המרה:")
print(df["גוף המסמך"].head())

df["גוף המסמך"] = df["גוף המסמך"].apply(clean_text)

print("\nשורות לאחר המרה:")
print(df["גוף המסמך"].head())

file_path_full = 'checked_df_cleaned.csv' # FULL RAP DATA CLEANED
df.to_csv(file_path_full, index=False, encoding='utf-8')


שורות לא קריאות לפני המרה:
5      ['החלטה בתיק רע"פ 3622/15 בבית המשפט העליון רע...
31     ['החלטה בתיק רע"פ 825/15 בבית המשפט העליון רע"...
67     ['החלטה בתיק רע"פ 54/15 בבית המשפט העליון רע"פ...
73     ['פסק דין בתיק רע"פ 2811/15 בבית המשפט העליון ...
114    ['פסק דין בתיק רע"פ 2195/15 בבית המשפט העליון ...
Name: גוף המסמך, dtype: object

שורות לאחר המרה:
5      ['החלטה בתיק רע"פ 3622/15 בבית המשפט העליון רע...
31     ['החלטה בתיק רע"פ 825/15 בבית המשפט העליון רע"...
67     ['החלטה בתיק רע"פ 54/15 בבית המשפט העליון רע"פ...
73     ['פסק דין בתיק רע"פ 2811/15 בבית המשפט העליון ...
114    ['פסק דין בתיק רע"פ 2195/15 בבית המשפט העליון ...
Name: גוף המסמך, dtype: object


In [17]:
def extract_relevant_part(text):
    words = str(text).split()
    if len(words) > 1000:
        return " ".join(words[-800:-20])
    return text

df["גוף המסמך חתוך"] = df["גוף המסמך"].apply(extract_relevant_part)

df[["גוף המסמך", "גוף המסמך חתוך"]].head()


,גוף המסמך,גוף המסמך חתוך
5,"['החלטה בתיק רע""פ 3622/15 בבית המשפט העליון רע...",מאסר תמורתו. 6. המבקש ערער לבית המשפט המחוזי ע...
31,"['החלטה בתיק רע""פ 825/15 בבית המשפט העליון רע""...","['החלטה בתיק רע""פ 825/15 בבית המשפט העליון רע""..."
67,"['החלטה בתיק רע""פ 54/15 בבית המשפט העליון רע""פ...","(13.1.2015); רע""פ 8087/14 נדים אנגנאץ נ\' מדינ..."
73,"['פסק דין בתיק רע""פ 2811/15 בבית המשפט העליון ...","['פסק דין בתיק רע""פ 2811/15 בבית המשפט העליון ..."
114,"['פסק דין בתיק רע""פ 2195/15 בבית המשפט העליון ...","['פסק דין בתיק רע""פ 2195/15 בבית המשפט העליון ..."


In [18]:
df = df[['גוף המסמך', 'מספר הליך', 'שם הליך', 'שנת פתיחת ההליך',
       'גוף המסמך חתוך', 'בקשה לרשות ערעור התקבלה?',
       'טקסט שמצביע על תוצאת בקשה שונה', 'הערעור התקבל?',
       'רע"פ בקשה אחרת או דלמטה מיוחד', 'הערות ', 'הערות קידוד חוזר',
       'טקסט שמצביע על תוצאת בקשה שונה (טקסטים שונים מופרדים ב-**)',
       'הערות תוצאת ערעור', 'length',]]

In [19]:
df.columns = df.columns.str.strip()
df.columns = df.columns.str.replace("\n", "")
df.columns = df.columns.str.replace("  ", " ")

In [20]:
df.rename(columns={
    'הערות ':"הערות",
    'טקסט שמצביע על תוצאת בקשה שונה':"טקסט שמצביע על תוצאת בקשה",
    "טקסט שמצביע על תוצאת בקשה שונה (טקסטים שונים מופרדים ב-**)": "טקסט שמצביע על התוצאה"
}, inplace=True)

df.columns


Index(['גוף המסמך', 'מספר הליך', 'שם הליך', 'שנת פתיחת ההליך',
       'גוף המסמך חתוך', 'בקשה לרשות ערעור התקבלה?',
       'טקסט שמצביע על תוצאת בקשה', 'הערעור התקבל?',
       'רע"פ בקשה אחרת או דלמטה מיוחד', 'הערות', 'הערות קידוד חוזר',
       'טקסט שמצביע על התוצאה', 'הערות תוצאת ערעור', 'length'],
      dtype='object')

In [21]:
columns_to_merge = [
    'טקסט שמצביע על תוצאת בקשה',
    'רע"פ בקשה אחרת או דלמטה מיוחד',  "הערות", 'הערות קידוד חוזר',
    "טקסט שמצביע על התוצאה",
    'הערות תוצאת ערעור'
]

def clean_merge_columns(row, columns):
    values = []
    for col in columns:
        if pd.notna(row[col]) and str(row[col]).strip():
            text = f"{str(row[col]).strip()}"
            text.replace("?", "")
            text
            text = re.sub(r"הערה של \S+\s*", "", text)
            values.append(text)

    return "\n".join(values) if values else None

# יצירת עמודה מאוחדת עם כותרות לכל ערך
df["תוצאה מסכמת"] = df.apply(lambda row: clean_merge_columns(row, columns_to_merge), axis=1)


# הצגת מספר דוגמאות לבדיקה
df[["תוצאה מסכמת"]].head()


,תוצאה מסכמת
5,לנוכח כל האמור לעיל – הבקשה למתן רשות ערעור נדחית
31,בקשה לעיכוב ביצוע עונש מאסר\nבקשה אחרת
67,"אשר על כן, הבקשה לרשות ערעור נדחית בזאת"
73,- זהו פסק דין משלים
114,חזרו בהם באי כוח המבקשים מהבקשות למתן רשות ערע...


In [22]:

for i in range(3):
    print(f"פסק דין {i+1} (תוצאה מסכמת):\n")
    print(df["תוצאה מסכמת"].iloc[i])
    print("="*80)


פסק דין 1 (תוצאה מסכמת):

לנוכח כל האמור לעיל – הבקשה למתן רשות ערעור נדחית
פסק דין 2 (תוצאה מסכמת):

בקשה לעיכוב ביצוע עונש מאסר
בקשה אחרת
פסק דין 3 (תוצאה מסכמת):

אשר על כן, הבקשה לרשות ערעור נדחית בזאת


In [23]:
pd.set_option('display.max_colwidth', None)

df[["גוף המסמך חתוך", "תוצאה מסכמת"]].sample(5, random_state=42)

,גוף המסמך חתוך,תוצאה מסכמת
3004,"['פסק דין בתיק רע""פ 4386/08 בבית המשפט העליון רע""פ 4386/08 בפני: כבוד השופטת א\' פרוקצ\'יה כבוד השופט א\' רובינשטיין כבוד השופט ס\' ג\'ובראן המבקש: גד כהן נגד המשיבה: מדינת ישראל בקשת רשות ערעור על פסק-דינו של בית המשפט המחוזי בירושלים מיום 30.4.08 בע""פ 2074/08 שניתן על-ידי כבוד השופט ע\' חבש – סג""נ בשם המבקש: עו""ד דוד גולן בשם המשיבה: עו""ד מיטל בוכמן-שינדל פסק דין לאור הסכמת הצדדים, המוצאת את ביטויה בסעיף 3 לתגובת המשיבה, ניתנת בזאת רשות ערעור, והערעור מתקבל במובן זה שפסקי הדין נשוא הבקשה שניתנו בעניינו של המבקש מבוטלים בזאת, והתיק מוחזר לבית משפט השלום לתעבורה בירושלים לדיון מחודש בעניינו. ניתן היום, א\' בחשון התשס""ט (30.10.2008). ש ו פ ט ת ש ו פ ט ש ו פ ט _________________________","ניתנת בזאת רשות ערעור, והערעור מתקבל במובן זה שפסקי הדין נשוא הבקשה שניתנו בעניינו של המבקש מבוטלים בזאת, והתיק מוחזר לבית משפט השלום לתעבורה"
1922,"['החלטה בתיק רע""פ 1682/11 בבית המשפט העליון רע""פ 1682/11 בפני: כבוד השופט ס\' ג\'ובראן המבקש: ואיל מזערה נגד המשיבה: מדינת ישראל בקשה לדחיית מועד תחילת ריצוי עונש המאסר בשם המבקש: עו""ד מאליק מנצור החלטה הבקשה נדחית. ניתנה היום, כ""ה באדר ב התשע""א (31.3.2011). ש ו פ ט _________________________",בקשה אחרת\nבקשה לדחיית מועד תחילת ריצוי עונש המאסר
3096,"['החלטה בתיק בש""פ 1092/17 בבית המשפט העליון רע""פ 1092/17 לפני: כבוד השופטת ד\' ברק-ארז המערערים: 1. כפיר דהן 2. שירה דהן נגד המשיבה: הוועדה המקומית לתכנון ובניה מטה יהודה בקשת רשות ערעור על פסק דינו של בית המשפט המחוזי בירושלים מיום 30.1.2017 בע""פ 23940-12-16 שניתן על-ידי כבוד השופט ח\' מ\' לומפ בשם המערערים: בעצמם בשם המשיבה: עו""ד רעות אהרון, עו""ד מאיר בן דוד פסק דין 1. הבקשה שבפני נקלטה במזכירות בית המשפט, בטעות, תחת הסיווג בש""פ. הלכה למעשה, מדובר בבקשת רשות לערער על פסק דינו של בית המשפט המחוזי בירושלים מיום 30.1.2017 (ע""פ 23940-12-16, השופטת ח\' מ\' לומפ). סיווגה הראוי אפוא הוא רע""פ, וכך אני מורה. 2. מקורו של ההליך דנן הוא בבקשה למתן אורכה לביצועו של צו ההריסה שהוצא ביחס למבנה היביל שבו מתגוררים המבקשים במסגרת הליך פלילי לפי חוק התכנון והבניה, התשכ""ה-1965. הבקשה נדחתה על ידי בית המשפט לעניינים מקומיים בבית שמש ועל כן הגישו המבקשים ערעור לבית המשפט המחוזי, אשר קבע כי תינתן למבקשים אורכה לפנות את המבנה עד ליום 1.4.2017 ולהרסו עד ליום 2.5.2017. לאחר מכן המבקשים הגישו בקשת אורכה נוספת לבית המשפט המחוזי, וזו נדחתה. על כך נסבה הבקשה שבפני. 3. בתשובה שהוגשה מטעם המשיבה צוין כי המקרה העומד ביסוד ההליך כרוך בנסיבות אישיות ומשפחתיות חריגות וקשות ועל כן היא מבקשת להותיר את ההכרעה בו לשיקול דעתו של בית המשפט. בהתחשב בכך ולאחר שבחנתי את מכלול הנסיבות העניין, אני מורה על מתן אורכה נוספת למבקשים לפנות את המבנה עד ליום 1.7.2017 ולהרסו עד ליום 2.8.2017 (בתנאים שנקבעו בפסק דינו של בית המשפט המחוזי). 4. הבקשה מתקבלת אפוא כמפורט בפסקה 3. על המבקשים להבין כי ההיענות לבקשתם התאפשרה לנוכח העמדה שגילתה המשיבה, וכי אל להם לצפות לכל הקלות נוספות, אלא להיערך לביצוע הצו השיפוטי, משתמו ההליכים כולם. ניתן היום, \u200fי""ט בשבט התשע""ז (\u200f15.2.2017). ש ו פ ט ת _________________________",ביהמ׳׳ש נתן אורכה אך לא ביטל את צו ההריסה\nהבקשה מתקבלת אפוא כמפורט בפסקה 3
3022,"['פסק דין בתיק רע""פ 1793/09 בבית המשפט העליון רע""פ 1793/09 בפני: כבוד השופט א\' א\' לוי כבוד השופטת ע\' ארבל כבוד השופט י\' דנציגר המבקשים: 1. פאבל מינסטר 2. זאב שוורץ נגד המשיבה: מדינת ישראל בקשת רשות ערעור על פסק-דינו של ביתה משפט המחוזי בתל-אביב, בע""פ 70149/08, מיום 28.1.09, שניתן על ידי כבוד ס\' הנשיא השופטת ד\' ברלינר, ס. הנשיא השופט ז\' המר, ס. הנשיא השופטת ב\' אופיר-תום בשם המבקש 1: עו""ד אמיר ציון בשם המבקש 2: עו""ד רובינשטיין מנחם פסק דין נוכח האמור בתגובת המשיבה, ניתנת בזאת רשות ערעור והבקשה תידון כבערעור גופו. עניינם של המבקשים יובא בשנית בפני בית המשפט המחוזי, המתבקש להבהיר אם תוצאת פסק-דינו היא זיכוי מוחלט של המבקשים או זיכוי מחמת הספק. ניתן היום, ט""ו באדר התשס""ט (11.03.09). ש ו פ ט ש ו פ ט ת ש ו פ ט _________________________",ביהמש לא הכריע בערעור והחזיר לדיון נוסף במחוזי
2649,"['בבית המשפט העליון בירושלים רע""פ 9608/01 בפני: כבוד השופט א\' ריבלין המבקשים: 1. גשמי ברכה ה.ש.י.ג.(1995) 2. גשמי ברכה בע""מ 3. ה.ש.י.ג.(1995)יזום פרוייקטים 4. שנהב 

In [24]:
df[["גוף המסמך חתוך", "תוצאה מסכמת"]].sample(5, random_state=42)

,גוף המסמך חתוך,תוצאה מסכמת
3004,"['פסק דין בתיק רע""פ 4386/08 בבית המשפט העליון רע""פ 4386/08 בפני: כבוד השופטת א\' פרוקצ\'יה כבוד השופט א\' רובינשטיין כבוד השופט ס\' ג\'ובראן המבקש: גד כהן נגד המשיבה: מדינת ישראל בקשת רשות ערעור על פסק-דינו של בית המשפט המחוזי בירושלים מיום 30.4.08 בע""פ 2074/08 שניתן על-ידי כבוד השופט ע\' חבש – סג""נ בשם המבקש: עו""ד דוד גולן בשם המשיבה: עו""ד מיטל בוכמן-שינדל פסק דין לאור הסכמת הצדדים, המוצאת את ביטויה בסעיף 3 לתגובת המשיבה, ניתנת בזאת רשות ערעור, והערעור מתקבל במובן זה שפסקי הדין נשוא הבקשה שניתנו בעניינו של המבקש מבוטלים בזאת, והתיק מוחזר לבית משפט השלום לתעבורה בירושלים לדיון מחודש בעניינו. ניתן היום, א\' בחשון התשס""ט (30.10.2008). ש ו פ ט ת ש ו פ ט ש ו פ ט _________________________","ניתנת בזאת רשות ערעור, והערעור מתקבל במובן זה שפסקי הדין נשוא הבקשה שניתנו בעניינו של המבקש מבוטלים בזאת, והתיק מוחזר לבית משפט השלום לתעבורה"
1922,"['החלטה בתיק רע""פ 1682/11 בבית המשפט העליון רע""פ 1682/11 בפני: כבוד השופט ס\' ג\'ובראן המבקש: ואיל מזערה נגד המשיבה: מדינת ישראל בקשה לדחיית מועד תחילת ריצוי עונש המאסר בשם המבקש: עו""ד מאליק מנצור החלטה הבקשה נדחית. ניתנה היום, כ""ה באדר ב התשע""א (31.3.2011). ש ו פ ט _________________________",בקשה אחרת\nבקשה לדחיית מועד תחילת ריצוי עונש המאסר
3096,"['החלטה בתיק בש""פ 1092/17 בבית המשפט העליון רע""פ 1092/17 לפני: כבוד השופטת ד\' ברק-ארז המערערים: 1. כפיר דהן 2. שירה דהן נגד המשיבה: הוועדה המקומית לתכנון ובניה מטה יהודה בקשת רשות ערעור על פסק דינו של בית המשפט המחוזי בירושלים מיום 30.1.2017 בע""פ 23940-12-16 שניתן על-ידי כבוד השופט ח\' מ\' לומפ בשם המערערים: בעצמם בשם המשיבה: עו""ד רעות אהרון, עו""ד מאיר בן דוד פסק דין 1. הבקשה שבפני נקלטה במזכירות בית המשפט, בטעות, תחת הסיווג בש""פ. הלכה למעשה, מדובר בבקשת רשות לערער על פסק דינו של בית המשפט המחוזי בירושלים מיום 30.1.2017 (ע""פ 23940-12-16, השופטת ח\' מ\' לומפ). סיווגה הראוי אפוא הוא רע""פ, וכך אני מורה. 2. מקורו של ההליך דנן הוא בבקשה למתן אורכה לביצועו של צו ההריסה שהוצא ביחס למבנה היביל שבו מתגוררים המבקשים במסגרת הליך פלילי לפי חוק התכנון והבניה, התשכ""ה-1965. הבקשה נדחתה על ידי בית המשפט לעניינים מקומיים בבית שמש ועל כן הגישו המבקשים ערעור לבית המשפט המחוזי, אשר קבע כי תינתן למבקשים אורכה לפנות את המבנה עד ליום 1.4.2017 ולהרסו עד ליום 2.5.2017. לאחר מכן המבקשים הגישו בקשת אורכה נוספת לבית המשפט המחוזי, וזו נדחתה. על כך נסבה הבקשה שבפני. 3. בתשובה שהוגשה מטעם המשיבה צוין כי המקרה העומד ביסוד ההליך כרוך בנסיבות אישיות ומשפחתיות חריגות וקשות ועל כן היא מבקשת להותיר את ההכרעה בו לשיקול דעתו של בית המשפט. בהתחשב בכך ולאחר שבחנתי את מכלול הנסיבות העניין, אני מורה על מתן אורכה נוספת למבקשים לפנות את המבנה עד ליום 1.7.2017 ולהרסו עד ליום 2.8.2017 (בתנאים שנקבעו בפסק דינו של בית המשפט המחוזי). 4. הבקשה מתקבלת אפוא כמפורט בפסקה 3. על המבקשים להבין כי ההיענות לבקשתם התאפשרה לנוכח העמדה שגילתה המשיבה, וכי אל להם לצפות לכל הקלות נוספות, אלא להיערך לביצוע הצו השיפוטי, משתמו ההליכים כולם. ניתן היום, \u200fי""ט בשבט התשע""ז (\u200f15.2.2017). ש ו פ ט ת _________________________",ביהמ׳׳ש נתן אורכה אך לא ביטל את צו ההריסה\nהבקשה מתקבלת אפוא כמפורט בפסקה 3
3022,"['פסק דין בתיק רע""פ 1793/09 בבית המשפט העליון רע""פ 1793/09 בפני: כבוד השופט א\' א\' לוי כבוד השופטת ע\' ארבל כבוד השופט י\' דנציגר המבקשים: 1. פאבל מינסטר 2. זאב שוורץ נגד המשיבה: מדינת ישראל בקשת רשות ערעור על פסק-דינו של ביתה משפט המחוזי בתל-אביב, בע""פ 70149/08, מיום 28.1.09, שניתן על ידי כבוד ס\' הנשיא השופטת ד\' ברלינר, ס. הנשיא השופט ז\' המר, ס. הנשיא השופטת ב\' אופיר-תום בשם המבקש 1: עו""ד אמיר ציון בשם המבקש 2: עו""ד רובינשטיין מנחם פסק דין נוכח האמור בתגובת המשיבה, ניתנת בזאת רשות ערעור והבקשה תידון כבערעור גופו. עניינם של המבקשים יובא בשנית בפני בית המשפט המחוזי, המתבקש להבהיר אם תוצאת פסק-דינו היא זיכוי מוחלט של המבקשים או זיכוי מחמת הספק. ניתן היום, ט""ו באדר התשס""ט (11.03.09). ש ו פ ט ש ו פ ט ת ש ו פ ט _________________________",ביהמש לא הכריע בערעור והחזיר לדיון נוסף במחוזי
2649,"['בבית המשפט העליון בירושלים רע""פ 9608/01 בפני: כבוד השופט א\' ריבלין המבקשים: 1. גשמי ברכה ה.ש.י.ג.(1995) 2. גשמי ברכה בע""מ 3. ה.ש.י.ג.(1995)יזום פרוייקטים 4. שנהב 

In [25]:
df.to_excel("/content/dataframe_cleaned.xlsx", index=False)

# split - train, validation, test

In [26]:
train_data, temp_data = train_test_split(df, test_size=0.2, random_state=42)
valid_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

def convert_to_jsonl(data, filename):
    with open(filename, "w", encoding="utf-8") as f:
        for _, row in data.iterrows():
            json.dump({
                "messages": [
                    {"role": "system", "content": "אתה עוזר משפטי לניתוח פסקי דין מסוג רע\"פ"},
                    {"role": "user", "content": row["גוף המסמך חתוך"]},
                    {"role": "assistant", "content": row["תוצאה מסכמת"]}
                ]
            }, f, ensure_ascii=False)
            f.write("\n")

convert_to_jsonl(train_data, "train.jsonl")
convert_to_jsonl(valid_data, "valid.jsonl")
convert_to_jsonl(test_data, "test.jsonl")


# GPT Fine tuning

In [27]:
with open("/content/api_key.txt", "r") as file:
    api_key = file.read().strip()


client = openai.OpenAI(api_key=api_key)



train_file = client.files.create(
    file=open("train.jsonl", "rb"),
    purpose="fine-tune"
)

print("Train File ID:", train_file.id)


Train File ID: file-7VVztU3Jp4qdfnAYiCR9q2


In [28]:
files = client.files.list()
print(files)

SyncCursorPage[FileObject](data=[FileObject(id='file-7VVztU3Jp4qdfnAYiCR9q2', bytes=1025228, created_at=1740495289, filename='train.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None, expires_at=None), FileObject(id='file-M37a1Mw35eyjq5tSTPt2xg', bytes=1023650, created_at=1740494945, filename='train.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None, expires_at=None), FileObject(id='file-2uDjhATy2M7N25mtwHgVFw', bytes=21008, created_at=1740490900, filename='step_metrics.csv', object='file', purpose='fine-tune-results', status='processed', status_details=None, expires_at=None), FileObject(id='file-53Ns892y8pFTfiYRrBJ8ki', bytes=1108947, created_at=1740489359, filename='train.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None, expires_at=None), FileObject(id='file-4uQENmlKIEQklPGwd7nccNbo', bytes=351964, created_at=1700573627, filename='COCKTAIL MENU EXAMLPLE.PNG', object='file', purpose='a

In [29]:
fine_tune = client.fine_tuning.jobs.create(
    training_file=train_file.id,
    model="gpt-3.5-turbo"
)

print("Fine-Tuning Job ID:", fine_tune.id)


Fine-Tuning Job ID: ftjob-Cgcdih0yQ80TUbwKrKftBITQ


In [30]:
fine_tune_status = client.fine_tuning.jobs.retrieve(fine_tune.id)
print(fine_tune_status)

FineTuningJob(id='ftjob-Cgcdih0yQ80TUbwKrKftBITQ', created_at=1740495299, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-XsklJyn5A831tAwBRgKN0tGP', result_files=[], seed=672308762, status='validating_files', trained_tokens=None, training_file='file-7VVztU3Jp4qdfnAYiCR9q2', validation_file=None, estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto')), type='supervised'), user_provided_suffix=None)


In [ ]:
job_id = fine_tune_status.id
while True:
    fine_tune_job = client.fine_tuning.jobs.retrieve(job_id)
    print(f"Status: {fine_tune_job.status}")

    if fine_tune_job.status in ["succeeded", "failed", "cancelled"]:
        print(f"Fine-Tuned Model ID: {fine_tune_job.fine_tuned_model}")
        break

    time.sleep(60*5)

Status: validating_files


In [ ]:
print(fine_tune_job.trained_tokens)
print(fine_tune_job.fine_tuned_model)

In [ ]:
result_files = client.files.list()
for file in result_files.data:
    if "fine-tuning" in file.purpose:
        print(file.filename, file.id)


In [ ]:
response = client.chat.completions.create(
    model=fine_tune_job.fine_tuned_model,
    messages=[
        {"role": "system", "content": 'אתה מומחה משפטי לניתוח ותמצות שורה תחתונה של פסקי דין מסוג רע"פ'},
        {"role": "user", "content": test_data.iloc[0]["גוף המסמך חתוך"]}
    ]
)

print("תשובה שהמודל נתן:")
print(response.choices[0].message.content)

print("תוצאה מקורית:\n")
print(test_data.iloc[0]["תוצאה מסכמת"])
